# Занятие №1. Параллельная обработка данных на чистом Python

На занятии с помощью Python мы воспроизвели паттерны, которые Apache Spark использует «под капотом». Программа, которая получилась в конце, читает большой файл с датасетом, фильтрует нужные события, группирует их в пакеты, после чего ообрабатывает параллельно в нескольких процессах. 

**Такой подход позволяет решить две проблемы:**
- файл с событиями может весить десятки гигабайт, если загрузить его целиком в список Python, программа упадёт с ошибкой `MemoryError` или начнёт использовать `swap`, что сильно замедлит работу;
- интепретатор Python выполняет байткод только в одном потоке одновременно, даже если в компьютере, например, восемь ядер, обычная однопоточная программа нагрузит только одно из них, а остальные семь будут простаивать.

Финальный вариант программа состоит из несколько логических блоков, по которым мы разделим наши действия.

**Всего таких блоков восемь:**
1. **Прописываем импорты и настраиваем окружение.** Прежде чем писать логику обработки, нужно настроить окружение. Программа использует несколько модулей стандартной библиотеки:

    - **functools** — содержит утилиты, которые позволяют работать с функциями. Из этого модуля нам понадобится декоратор `wraps`, который сохраняет метаданные функции, которую мы будем оборачивать;
    - **json** — парсит JSON-строки в словари Python;
    - **logging** — выводит сообщения с информацией о том, как выполняется программа, логирование поможет нам понять, что делает конкретный процесс;
    - **time** — измеряет время, за которое выполняются функции;
    - **collections.abc** — содержит абстрактные типы `Iterable` и `Iterator` для аннотаций, которые мы будем использовать;
    - **concurrent.futures** — содержит высокоуровневый интерфейс, который позволяет запускать задачи параллельно в нескольких процессах или потоках;
    - **pathlib** — предоставляет пути к файлам как к объектам, а не строкам.

Константа `WORKDIR` хранит путь к директории, где лежит скрипт. От неё мы будем строить относительные пути к данным.

Сам логгер мы настраиваем так, чтобы каждое сообщение содержало имя процесса. Когда мы запустим несколько процессов параллельно, то сразу увидим, какой из них пишет в лог.

In [ ]:
# Фрагмент программы №1

import functools
import json
import logging
import time
from collections.abc import Iterable, Iterator
from concurrent.futures import ProcessPoolExecutor, as_completed
from pathlib import Path

WORKDIR = Path(__file__).parent

logging.basicConfig(level=logging.INFO, format="%(processName)s %(message)s")
logger = logging.getLogger(__name__)


2. **Используем декоратор `slowlog`**
    
    **Декоратор** — это функция, которая принимает другую функцию и возвращает её модифицированную версию. Декораторы позволяют добавить поведение к функции, но при этом не менять её код.

    Декоратор `slowlog` измеряет, сколько времени выполняется функция, и пишет результат в лог. Параметр `threshold` задаёт порог в секундах. Если функция выполняется дольше, чем этот порог, то декоратор помечает её в логе, что она слишком медленная.

    Кроме того, здесь используется **фабрика декораторов** — функция, которая возвращает декоратор. Такая конструкция нужна, чтобы передать параметр `threshold`. 

    **Давайте разберём её структуру:**
    1. **`slowlog(threshold)`** — это внешняя функция, которая принимает параметр `threshold` и возвращает декоратор `set_timer`.
    2. **`@functools.wraps(func)`** — это декоратор, который копирует в обёртку метаданные оригинальной функции. Например, имя и документацию. Без него `wrapper.__name__` вернул бы просто `"wrapper"` вместо настоящего имени функции.
    3. **`set_timer(func)`** — это сам декоратор, который принимает функцию и возвращает обёртку `wrapper`.
    4. **`wrapper(*args, **kwargs)`** — это обёртка, которая вызывает оригинальную функцию и логирует результат.
    5. **`time.perf_counter()`** — это функция, возвращает время с максимальной точностью. Она лучше подходит, чтобы измерять короткие интервалы, чем `time.time()`.
    6. **`try/finally`** — это блок, который гарантирует, что время запишется в лог даже если функция выбросит исключение.

In [ ]:
#Фрагмент программы №2

def slowlog(threshold: float = 2.5):
    def set_timer(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            start = time.perf_counter()
            try:
                result = func(*args, **kwargs)
            finally:
                delta = time.perf_counter() - start
                if delta > threshold:
                    logger.info("Finished %s in %.3f seconds (too slow)", func.__name__, delta)
                else:
                    logger.info("Finished %s in %.3f seconds", func.__name__, delta)
            return result
        return wrapper
    return set_timer
    

3. **Используем генератор `read_events`.**

    **Генератор** — это функция, которая использует ключевое слово `yield` вместо `return`. Генератор не выполняет всю работу сразу, он возвращает объект-итератор, который выдаёт по одному значению при каждом обращении.

    Функция `read_events` читает файл в формате **JSONL, или JSON Lines,** — это формат, в котором каждая строка файла является отдельным JSON-объектом. Такой формат удобно использовать для потоковой обработки, так как нам не нужно парсить весь файл целиком, чтобы получить первую запись.

    Генератор открывает файл и проходит по нему построчно. Каждую строку он парсит в словарь и отдаёт через `yield`. Важно отметить, что пока внешний код не запрос следующее значение, генератор стоит на паузе и не читает следующую строку. Такой подход называется **ленивым вычислением**. Он позволяет сэкономить оперативную память, так как в каждый момент памяти находится только одна строка файла, а не весь файл целиком.

    Аннотация `Iterator[dict]` говорит нам, что функция возвращает итератор, который выдаёт словари.

In [ ]:
# Фрагмент программы №3

def read_events(path: Path) -> Iterator[dict]:
    with open(path, "r", encodiing="utf-8") as f:
        for line in f:
            yield json.loads(line)
            

4. **Используем генератор `filter_events`.**

    Функция `filter_events` принимает итератор событий и возвращает новый итератор, который пропускает только события с нужным типом. Этот генератор тоже ленивый, так как он не создаёт список отфильтрованных событий в памяти. Вместо этого он берёт события из входного итератора по одному, проверяет условие и отдаёт подходящие дальше.

    Когда мы соединяем `read_events` и `filter_events` в цепочку, то получаем **пайплайн** — это последовательность преобразований, где каждое звено обрабатывает данные и передаёт следующие. Важно отметить, что при этом ни одно звено не хранит данные целиком.

In [ ]:
# Фрагмент программы №4

def filter_events(events: Iterator[dict], wanted_event_type: str) -> Iterator[dict]:
    for event in events:
        if event["event_type"] == wanted_event_type:
            yield event
            

5. **Группируем потоки в пакеты.**

    **Батч** — это пакет элементов фиксированного размера.

    Когда программа передаёт данные в другой процесс, она сначала сериализует их, или превращает в байты, после чего отправляет, а процесс-получатель десереализует эти данные обратно. Эти операции занимают время. Если отправлять события по одному, то расходы на передачу данные перекроют время, которые мы получили благодаря параллельной обработки. Крупные пакеты выгоднее передавать, так как мы передаём один большой вместо тысяч маленьких.

    Функция `batcher` принимает итератор и размер батча. Она складывает элементы в список. Как только список достигает нужного размера, `batcher` отдаёт его через `yield` и начинает собирать следующий. После того, как цикл завершиться, в конце файла элементов может быть меньше, чем размер батча. Для этого мы используем условие `if batch`, чтобы проверить, так ли это, и отдать остаток элементов последним неполным пакетом.

    Аннтоация `Iterable[list[dict]]` говорит нам, что функция возвращает итерируемый объект, который выдаёт списки словарей.

In [ ]:
# Фрагмент программы №5

def batcher(iterable: Iterable[dict], batch_size: шт) -> Iterable[list[dict]]:
    batch = []
    for item in iterable:
        batch.append(item)
        if len(batch) == batch_size:
            yield batch
            batch = []
        if batch:
            yield batch
            

6. **Обрабатываем батч и имитируем CPU-bound операцию.**